In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [19]:
#trainフォルダ内のcsvファイルを全て取得
files = glob.glob("train/*.csv")

data_list = []
#filesに格納された４７個のファイルをリストに入れる
for file in files:
    data_list.append(pd.read_csv(file,index_col=0))
#追加したリストデータの結合
df = pd.concat(data_list)

In [20]:
def data_pre(df):
    nonnull_list = []

    #カラムを呼び出しnull数を計算、０のものをnonnull_listに追加する
    for col in df.columns:
        nonnull = df[col].count()
        if nonnull == 0:
            nonnull_list.append(col)
            
    df = df.drop(nonnull_list,axis=1) 

    df = df.drop(["市区町村名","種類"],axis=1) 

    dis = {
        "30分?60分":45,
        "1H?1H30":75,
        "2H?":120,
        "1H30?2H":105
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)
    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上",2000).astype(float)
    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[1].split("年")[0])
            year = 34 - num
        if "令和" in i: 
            num = float(i.split("令和")[1].split("年")[0])
            year = 4 - num
        if "昭和" in i:
            num = float(i.split("昭和")[1].split("年")[0])
            year = 97 - num
        y_list[i] = year
    df["建築年"] = df["建築年"].replace(y_list)
    df["取引時点"]
    year = {
        "年第1四半期":"0.25",
        "年第2四半期":"0.5",
        "年第3四半期":"0.75",
        "年第4四半期":"0.99"
    }

    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k, j)
        year_list[i] = year_rep

    df["取引時点"]=df["取引時点"].replace(year_list).astype(float)
    
    return df

In [21]:
df = data_pre(df)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707485 entries, 43046834 to 45025344
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   市区町村コード       707485 non-null  int64  
 1   都道府県名         707485 non-null  object 
 2   地区名           706818 non-null  object 
 3   最寄駅：名称        704776 non-null  object 
 4   最寄駅：距離（分）     684350 non-null  float64
 5   間取り           682328 non-null  object 
 6   面積（㎡）         707485 non-null  float64
 7   建築年           686959 non-null  float64
 8   建物の構造         689270 non-null  object 
 9   用途            640702 non-null  object 
 10  今後の利用目的       342163 non-null  object 
 11  都市計画          687864 non-null  object 
 12  建ぺい率（％）       683596 non-null  float64
 13  容積率（％）        683596 non-null  float64
 14  取引時点          707485 non-null  float64
 15  改装            641456 non-null  object 
 16  取引の事情等        18824 non-null   object 
 17  取引価格（総額）_log  707485 non-null  float64
